In [1]:
import sympy as sp

sMul = sp.Mul

from symqups import s as CahillGlauberS
from symqups import MP_CONFIG
from symqups.objects.scalars import (q, p, alpha, alphaD, W, mu, _Primed, 
                                     _DerivativeSymbol, hbar, _deprime, StateFunction)
from symqups.objects.operators import qOp, pOp, annihilateOp, createOp, rho, Operator
from symqups.manipulations import define, qp2a, explicit, express
from symqups.ordering import sOrdering
from symqups.quantization import s_quantize, naive_quantize
from symqups.cg import CG_transform, iCG_transform
from symqups.objects import scalars
from symqups._internal.cache import sub_cache
from symqups.star_product import Bopp, Star, _replace_diff
from symqups._internal.basic_routines import operation_routine
from symqups._internal.operator_handling import (
    separate_operator,
    separate_term_by_polynomiality,
    separate_term_oper_by_sub,
    collect_alpha_type_oper_from_monomial_by_sub,
    get_oper_sub
)
from symqups._internal.warnings import suppress_warning_repeats
from symqups._internal.grouping import UnBoppable, NotAnOperator, PhaseSpaceObject
from symqups.utils import get_random_poly
from symqups.eom import LindbladMasterEquation

a = [alpha(i) for i in range(10)]
aop = [annihilateOp(i) for i in range(10)]
ad = [alphaD(i) for i in range(10)]
adop = [createOp(i) for i in range(10)]
x = sp.Symbol("x")
qq = [q(i) for i in range(10)]
pp = [p(i) for i in range(10)]
W = scalars.W
CahillGlauberS.val = sp.Number(0.3)
scalars.mu = 1
scalars.hbar = 1

In [ ]:
LindbladMasterEquation(H = adop[0]*aop[0]).rhs.doit().expand()

0